<a href="https://colab.research.google.com/github/Tom-Jung/legendary-funicular/blob/main/Ch6_2.%20%EC%82%B0%EC%B6%9C%20%EB%B3%80%EB%8F%99%EC%9A%94%EC%9D%B8%20%EB%B6%84%ED%95%B4_3%EB%B6%80%EB%AC%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np   # Numpy import
import pandas as pd  # Pandas import

In [44]:
np.set_printoptions(precision=3,suppress=True) # 결과를 소수 3자리까지만 보여주는 옵션

### 산출 변동요인분해

In [45]:
# 2015년 거래표 다운로드

url_total = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Total_Matrix_2015_cst.csv"  # 불변 2015년 총거래표
Tot_cst_15 = pd.read_csv(url_total)

url_imp = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Imported_Matrix_2015_cst.csv"  # 불변 2015년 수입거래표
Imp_cst_15 = pd.read_csv(url_imp)

url_dom = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Domestic_Matrix_2015_cst.csv"  # 불변 2015년 국산거래표
Dom_cst_15 = pd.read_csv(url_dom)

In [46]:
# 2010년 거래표 다운로드

url_total = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Total_Matrix_2010_cst.csv"  # 불변 2010년 총거래표
Tot_cst_10 = pd.read_csv(url_total)

url_imp = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Imported_Matrix_2010_cst.csv"  # 불변 2010년 수입거래표
Imp_cst_10 = pd.read_csv(url_imp)

url_dom = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Domestic_Matrix_2010_cst.csv"  # 불변 2010년 국산거래표
Dom_cst_10 = pd.read_csv(url_dom)

In [47]:
Tot_cst_10.shape

(41, 47)

부문통합하여 3부문 IO표 만들기

In [48]:
cdd = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/code.csv"  # 코드매치 파일
ccdd = pd.read_csv(cdd)
cd_for_itr = ccdd.to_numpy().reshape(33,2)
cd_3 = cd_for_itr[:,1]
cd_33 = cd_for_itr[:,0]
print('3부문 코드 : \n', cd_3)
print('33부문 코드 : \n', cd_33)

3부문 코드 : 
 [1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
33부문 코드 : 
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33]


In [49]:
# Mtx = np.zeros((3,32)) # 부문통합을 위한 코드가 많을 경우 상기의 코드를 수작업으로 미리 하는 것이 쉽자 않으므로 반복문을 통해 3x33 임시코드 만들기
# row=cd_3[:-2]
# col=cd_33[:-2]
# for i, j in zip(row,col):
#   Mtx[i-1,j-1] = 1
# print(Mtx)

In [89]:
code = np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],  # 행렬 곱을 이용한 부문 통합을 위해 0과 1로 구성된 임시의 3x33 매트릭스
       [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]])
code = code.astype(float)

In [118]:
SS_t0 = (Tot_cst_10.iloc[1:41,2:].to_numpy().astype(float))/1000000 # 10년 총거래표 numpy로 숫자만 불러오기(시간 t-1)
II_t0 = (Imp_cst_10.iloc[1:38,2:].to_numpy().astype(float))/1000000 # 10년 수입거래표 numpy로 숫자만 불러오기

SS_t5 = (Tot_cst_15.iloc[1:41,2:].to_numpy().astype(float))/1000000 # 15년 총거래표 numpy로 숫자만 불러오기(시간 t)
IS_t5 = (Imp_cst_15.iloc[1:38,2:].to_numpy().astype(float))/1000000 # 15년 수입거래표 numpy로 숫자만 불러오기


In [119]:
nae_tot_row_t0 = code@SS_t0[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기
nae_tot_row_t5 = code@SS_t5[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기

In [120]:
nae_tot_col_t0 = nae_tot_row_t0@code.T  # 중간투입(수요)의 열을 3부문으로 만들기
nae_tot_col_t5 = nae_tot_row_t5@code.T  # 중간투입(수요)의 열을 3부문으로 만들기

In [122]:
nae_tot_col_t5

array([[  3.225,  39.057,  11.897],
       [ 17.751, 925.393, 350.855],
       [  7.137, 261.663, 575.308]])

In [123]:
va_row_t0 = SS_t0[32:42,0:32]@code.T # 중투합, 피용자보수, 영업잉여, 고정자본소모, 기타생산세, 조정항, 부가가치계, 산출액(8X3)
va_row_t5 = SS_t5[32:41,0:32]@code.T # 중투합, 피용자보수, 영업잉여, 고정자본소모, 기타생산세, 조정항, 부가가치계, 산출액(7X3)
va_row_t5

array([[  28.113, 1226.114,  938.06 ],
       [   5.243,  176.815,  568.155],
       [  22.576,  139.967,  278.922],
       [   4.828,  114.82 ,  190.021],
       [   0.61 ,   53.438,   82.055],
       [  33.258,  485.04 , 1119.152],
       [  61.372, 1711.154, 2057.213]])

In [124]:
fd_cd = np.array([[1,1,0,0,0,0,0,0,0],   # 최종수요를 통합하기 위한 0과 1로된 행렬
                  [0,0,1,1,1,1,0,0,0],
                  [0,0,0,0,0,0,1,0,0],
                  [0,0,0,0,0,0,0,1,0],
                  [0,0,0,0,0,0,0,0,1]])


In [125]:
fd_t0 = SS_t0[0:32, 33:42]
fd_t5 = SS_t5[0:32, 33:42]

In [126]:
fd_row_t0=code@fd_t0@fd_cd.T
fd_row_t5=code@fd_t5@fd_cd.T
fd_row_t0

array([[  16.01 ,    2.646,    0.83 ,   19.485,   68.904],
       [ 166.514,  137.44 ,  456.483,  760.437, 1847.788],
       [ 740.29 ,  310.398,   78.546, 1129.235, 1819.324]])

In [127]:
fd_row_sm_t0 = np.sum((fd_row_t0), axis=0) # 최종수요의 행합
fd_row_sm_t5 = np.sum((fd_row_t5), axis=0) # 최종수요의 행합

In [130]:
fd_sm_t0=fd_row_sm_t0.reshape(1,5)
fd_sm_t5=fd_row_sm_t5.reshape(1,5)

In [132]:
nae_va_ct_t0 = np.concatenate((nae_tot_col_t0, va_row_t0)) # 중간투입(수요)와 부가가치 및 산출액 합치기
nae_va_ct_t5 = np.concatenate((nae_tot_col_t5, va_row_t5)) # 중간투입(수요)와 부가가치 및 산출액 합치기

In [134]:
nae_va_ct_sm_t0 = nae_va_ct_t0.sum(axis=1).reshape(11,1)  # 부가가치 및 산출액의 열합
nae_va_ct_sm_t5 = nae_va_ct_t5.sum(axis=1).reshape(10,1)  # 부가가치 및 산출액의 열합

In [135]:
nnull_t0 = np.zeros((7,5)) # 0행렬
nnull_t5 = np.zeros((6,5)) # 0행렬

In [137]:
fd_nnull_t0 = np.concatenate((fd_row_t0, fd_sm_t0, nnull_t0),axis=0)  # 최종수요, 최종수요 행합 및 0행렬 합치기
fd_nnull_t5 = np.concatenate((fd_row_t5, fd_sm_t5, nnull_t5),axis=0)  # 최종수요, 최종수요 행합 및 0행렬 합치기

In [140]:
tot_f_t0 = np.concatenate((nae_va_ct_t0,nae_va_ct_sm_t0, fd_nnull_t0), axis=1)  # 최종 3x3 부문통합된 거래표
tot_f_t5 = np.concatenate((nae_va_ct_t5,nae_va_ct_sm_t5, fd_nnull_t5), axis=1)  # 최종 3x3 부문통합된 거래표

In [145]:
tot_F_t0 = pd.DataFrame(tot_f_t0.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계', '피용자보수','영업잉여', '고정자본소모', '생산세(보조금공제)','조정항', '부가가치계', '총투입액']) # 변수명 붙여서 pandas로 내보내기


In [146]:
tot_F_t5 = pd.DataFrame(tot_f_t5.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계', '피용자보수','영업잉여', '고정자본소모', '생산세(보조금공제)','부가가치계', '총투입액']) # 변수명 붙여서 pandas로 내보내기


In [148]:
# 2010, 2015년 3부문 불변표 내보내기
tot_F_t0.to_csv('10년_총거래표_3부문_불변.csv', encoding="utf-8-sig")
tot_F_t5.to_csv('15년_총거래표_3부문_불변.csv', encoding="utf-8-sig")

In [88]:
print(nae_tot_row)

[[4.544e+06 3.784e+03 3.273e+07 7.005e+05 1.020e+06 1.269e+03 1.097e+06
  3.216e+03 9.706e+03 4.920e+03 1.288e+04 3.040e+03 8.898e+03 1.352e+04
  2.360e+04 3.882e+04 2.048e+03 3.434e+03 9.738e+05 6.285e+04 1.553e+04
  6.717e+06 2.754e+04 2.882e+04 5.006e+03 1.142e+05 1.327e+04 5.897e+04
  4.491e+04 1.004e+06 1.005e+05 3.624e+04]
 [1.777e+07 6.033e+05 2.481e+07 3.982e+07 1.853e+07 5.414e+07 1.067e+08
  1.537e+07 6.826e+07 4.195e+07 1.335e+08 4.994e+07 6.478e+07 1.329e+08
  8.858e+06 1.304e+07 3.363e+07 4.896e+06 8.780e+07 1.409e+07 3.224e+07
  3.223e+07 1.158e+07 5.403e+06 1.354e+06 1.761e+07 4.333e+06 6.540e+06
  6.914e+06 2.173e+07 5.104e+06 1.096e+07]
 [5.781e+06 1.030e+06 1.763e+07 1.369e+07 8.046e+06 4.932e+06 3.270e+07
  8.693e+06 2.486e+07 1.191e+07 3.137e+07 1.137e+07 1.551e+07 2.939e+07
  3.415e+06 5.724e+06 2.325e+07 6.630e+06 3.573e+07 7.032e+07 4.650e+07
  2.300e+07 4.145e+07 5.241e+07 2.725e+07 5.113e+07 1.182e+07 2.008e+07
  1.703e+07 1.725e+07 1.130e+07 8.888e+06]]


In [19]:
Tot_33_15 = Tot_cst_15.to_numpy()
Imp_33_15 = Imp_cst_15.to_numpy()
Dom_33_15 = Dom_cst_15.to_numpy()

Tt_33_15 = Tot_33_15[1:40,2:47].astype(float)
Im_33_15 = Imp_33_15[1:40,2:47].astype(float)
Dm_33_15 = Dom_33_15[1:40,2:47].astype(float)


array([[ 3.225e+06,  5.492e+03,  3.549e+07, ...,  6.137e+07,  1.216e+07,
         7.353e+07],
       [ 1.031e+03,  2.070e+03,  2.650e+04, ...,  4.125e+06,  1.122e+08,
         1.163e+08],
       [ 9.277e+06,  8.905e+03,  2.320e+07, ...,  1.221e+08,  2.619e+07,
         1.483e+08],
       ...,
       [ 6.100e+05, -6.183e+04,  1.185e+07, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 3.326e+07,  2.138e+06,  3.065e+07, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 6.137e+07,  4.125e+06,  1.221e+08, ...,  0.000e+00,  0.000e+00,
         0.000e+00]])

In [ ]:
S_t0 = Tot_cst_10.iloc[1:41,2:].to_numpy() # 10년 총거래표 numpy로 숫자만 불러오기(시간 t-1)
I_t0 = Imp_cst_10.iloc[1:38,2:].to_numpy() # 10년 수입거래표 numpy로 숫자만 불러오기

S_t5 = Tot_cst_15.iloc[1:41,2:].to_numpy() # 15년 총거래표 numpy로 숫자만 불러오기(시간 t)
I_t5 = Imp_cst_15.iloc[1:38,2:].to_numpy() # 15년 수입거래표 numpy로 숫자만 불러오기

In [ ]:
S1 = S_t5.astype(float)
I1 = I_t5.astype(float)
S0 = S_t0.astype(float)
I0 = I_t0.astype(float)

In [ ]:
SS1 = np.concatenate((S1[:37,],np.zeros((1,45)), S1[37:39,] ), axis=0)  # 2015년 부가가치 조정항을 0으로 삽입(2010년과 동일한 포맷을 유지하기 위함)
print(S_t0.shape)
print(S_t5.shape)

(40, 45)
(39, 45)


In [ ]:
SS0 = S0[0:32,0:32].astype(float)  # 내생항목 32x32
SS1 = S1[0:32,0:32].astype(float)
SS1.shape

(32, 32)

In [ ]:
II0 = I0[0:32,0:32]  # 수입 내생항목 32x32
II1 = I1[0:32,0:32]

In [ ]:
DD0 =SS0 - II0  # 국산 내생항목 32x32
DD1 =SS1 - II1

In [ ]:
TM0 = I0[0:32,41]  # 총수입벡터
TM1 = I1[0:32,41]
TM0.shape

(32,)

In [ ]:
IM0 = I0[0:32,32]  # 중간재수입벡터
IM1 = I1[0:32,32]

In [ ]:
IF0 = I0[0:32,40]  # 최종재수입벡터
IF1 = I1[0:32,40]
IF1

array([ 2841675.,   -78991., 11200552., 14576720.,   416107.,  1440129.,
        5774500.,   241793.,  -375737.,  1468072., 21654896.,  4476095.,
       21956549., 20812067.,  5102279.,       80.,    65851.,    16766.,
          16539.,        0.,  6246877., 11680866.,  2267143.,   704737.,
         589893.,  5921055.,   345575.,        0.,  2322148.,   766938.,
        4040457.,   268235.])

In [ ]:
SFC0 = S0[0:32,33:35]  # 최종수요-소비
SFC0 = SFC0.sum(axis=1)
SFC1 = S1[0:32,33:35]
SFC1 = SFC1.sum(axis=1)
SFC1

array([ 1.701e+07,  1.305e+04,  5.674e+07,  3.036e+07,  1.257e+06,
        1.473e+07,  8.106e+06,  2.421e+05, -5.563e+05,  1.322e+06,
        2.491e+07,  5.265e+06,  1.411e+06,  3.125e+07,  7.145e+06,
        2.359e+04,  1.775e+07,  4.581e+06,  0.000e+00,  7.896e+07,
        2.587e+07,  8.281e+07,  3.253e+07,  6.554e+07,  1.148e+08,
        3.138e+06,  3.895e+06,  1.210e+08,  1.163e+08,  1.129e+08,
        3.544e+07,  4.021e+07])

In [ ]:
SFI0 = S0[0:32,35:39 ] # 최종수요-투자
SFI0 = SFI0.sum(axis=1)
SFI1 = S1[0:32,35:39]
SFI1 = SFI1.sum(axis=1)
SFI1

array([ 1.655e+06, -1.248e+05,  5.540e+04,  1.122e+06,  1.029e+05,
        5.121e+05, -6.173e+05, -1.789e+04, -4.026e+06,  6.691e+06,
        2.623e+07,  1.163e+07,  5.386e+07,  3.807e+07,  4.760e+06,
        0.000e+00,  0.000e+00,  0.000e+00,  2.121e+08,  1.331e+07,
        3.939e+05,  0.000e+00,  2.835e+07,  0.000e+00,  2.603e+07,
        6.905e+07,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        5.053e+05,  0.000e+00])

In [ ]:
SF0 = S0[0:32,40] # 최종수요합계
SF1 = S1[0:32,40]
SF1

array([1.935e+07, 1.746e+04, 6.433e+07, 6.652e+07, 4.954e+06, 5.551e+07,
       9.037e+07, 3.113e+06, 3.494e+07, 1.990e+07, 2.253e+08, 4.647e+07,
       1.032e+08, 1.990e+08, 1.477e+07, 2.812e+06, 1.787e+07, 4.964e+06,
       2.122e+08, 1.164e+08, 6.422e+07, 8.986e+07, 6.641e+07, 6.839e+07,
       1.412e+08, 8.809e+07, 1.268e+07, 1.210e+08, 1.165e+08, 1.132e+08,
       3.698e+07, 4.082e+07])

In [ ]:
EX0 = S0[0:32,39] # 수출
EX1 = S1[0:32,39]
EX1

array([6.871e+05, 1.292e+05, 7.536e+06, 3.504e+07, 3.594e+06, 4.027e+07,
       8.288e+07, 2.889e+06, 3.952e+07, 1.189e+07, 1.742e+08, 2.958e+07,
       4.791e+07, 1.297e+08, 2.868e+06, 2.788e+06, 1.167e+05, 3.830e+05,
       1.651e+05, 2.413e+07, 3.795e+07, 7.042e+06, 5.528e+06, 2.859e+06,
       4.298e+05, 1.590e+07, 8.785e+06, 3.756e+04, 1.712e+05, 3.703e+05,
       1.037e+06, 6.119e+05])

In [ ]:
DF0 = SF0 - EX0 #소비+투자
DF1 = SF1 - EX1

In [ ]:
SFC00 = SFC0.sum(axis=0) # 소비합계
SFC11 = SFC1.sum(axis=0)

SFI00 = SFI0.sum(axis=0) # 투자합계
SFI11 = SFI1.sum(axis=0)

EX00 = EX0.sum(axis=0) # 수출합계
EX11 = EX1.sum(axis=0)

print('2010년 소비합계 : ', SFC00)
print('2015년 소비합계 : ', SFC11)

print('2010년 투자합계 : ', SFI00)
print('2015년 투자합계 : ', SFI11)

print('2010년 수출합계 : ', EX00)
print('2015년 수출합계 : ', EX11)


2010년 소비합계 :  922814206.0
2015년 소비합계 :  1054900320.0
2010년 투자합계 :  450484823.0
2015년 투자합계 :  489601534.0
2010년 수출합계 :  535858572.0
2015년 수출합계 :  716951177.0


In [ ]:
GDC1 = ((SFC11 - SFC00)/ SFC00) # 소비증가율
GDC2 = GDC1*SFC0  # 규모효과
GDC3 = (SFC1-SFC0) - GDC2 # 구조효과

In [ ]:
GDI1 = ((SFI11 - SFI00)/ SFI00) # 투자증가율
GDI2 = GDI1*SFI0 # 규모효과
GDI3 = (SFI1-SFI0) - GDI2 # 구조효과

In [ ]:
ED1 = ((EX11 - EX00)/EX00) # 수출증가율
ED2 = ED1*EX0 # 규모효과
ED3 = (EX1-EX0) - ED2 # 구조효과

In [ ]:
C0 = S0[-1,0:32].T.reshape([32,1])  # 총산출액
C1 = S1[-1,0:32].T.reshape([32,1])
C1.shape

(32, 1)

In [ ]:
TC0 = C0.sum(axis=0)  #총산출계
TC1 = C1.sum(axis=0)

In [ ]:
V0 = S0[-1,0:32].T - S0[32,0:32].T # 부가가치계벡터 = 총투입액 - 중간투입계
V1 = S1[-1,0:32].T - S1[32,0:32].T
V1

array([3.326e+07, 2.138e+06, 3.065e+07, 1.535e+07, 1.337e+07, 3.389e+07,
       7.084e+07, 1.246e+07, 2.518e+07, 3.597e+07, 9.392e+07, 2.918e+07,
       3.676e+07, 4.964e+07, 5.631e+06, 3.008e+07, 3.414e+07, 1.026e+07,
       9.353e+07, 1.354e+08, 5.488e+07, 4.555e+07, 7.302e+07, 9.223e+07,
       1.464e+08, 9.501e+07, 5.343e+07, 9.549e+07, 8.042e+07, 6.314e+07,
       2.222e+07, 2.405e+07])

In [ ]:
AV0 = V0/C0 # 부가가치계수
AV1 = V1/C1

In [ ]:
CT0 = np.tile(C0, reps=[1,32])   # Ct벡터를 반복해여 행렬(32x32)로 만듬
CT1 = np.tile(C1, reps=[1,32])
CT1

array([[6.137e+07, 6.137e+07, 6.137e+07, ..., 6.137e+07, 6.137e+07,
        6.137e+07],
       [4.125e+06, 4.125e+06, 4.125e+06, ..., 4.125e+06, 4.125e+06,
        4.125e+06],
       [1.221e+08, 1.221e+08, 1.221e+08, ..., 1.221e+08, 1.221e+08,
        1.221e+08],
       ...,
       [1.196e+08, 1.196e+08, 1.196e+08, ..., 1.196e+08, 1.196e+08,
        1.196e+08],
       [4.029e+07, 4.029e+07, 4.029e+07, ..., 4.029e+07, 4.029e+07,
        4.029e+07],
       [5.419e+07, 5.419e+07, 5.419e+07, ..., 5.419e+07, 5.419e+07,
        5.419e+07]])

In [ ]:
AS0 = SS0/CT0.T # 총투입계수표
AS1 = SS1/CT1.T
# np.isnan(AS_t)

In [ ]:
AD0 = DD0/CT0.T # 국산투입계수표
AD1 = DD1/CT1.T

In [ ]:
II0 = AS0-AD0 # 수입투입계수표
II1 = AS1-AD1

In [ ]:
M0 = IF0/DF0 # 수입최종수요/(소비+투자)
M0 = np.diag(M0)

M1 = IF1/DF1
M1 = np.diag(M1)

In [ ]:
RD0 = np.linalg.inv(np.eye(32) - AD0) # 생산유발계수
RD1 = np.linalg.inv(np.eye(32) - AD1)

In [ ]:
AM0 = II0/AS0 #조정항목
AM0 = AM0*AS1

AM1 = II1/AS1
AM1 = AM1*AS0

np.nan_to_num(AM0, copy=False)
np.nan_to_num(AM1, copy=False)

<ipython-input-109-5a495a19eb92>:1: RuntimeWarning: invalid value encountered in true_divide
  AM0 = II0/AS0 #조정항목
<ipython-input-109-5a495a19eb92>:4: RuntimeWarning: invalid value encountered in true_divide
  AM1 = II1/AS1


array([[0.008, 0.   , 0.052, ..., 0.001, 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.003, 0.   , 0.052, ..., 0.002, 0.001, 0.   ],
       ...,
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.001, 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ]])

In [ ]:
AA = AS1 - AS0 #투입계수변화
AS1_f = pd.DataFrame(AS1)
AS1_f.to_csv('AS1_f.csv')

#### 산출액성장요인분해
-  파쉐방식으로 산출액 변동을 분해

In [ ]:
TC1 = ((RD1@(np.eye(32)-M1))@GDC2).reshape([32,1])  # 소비규모증가효과
TC11 = ((RD1@(np.eye(32)-M1))@GDC3).reshape([32,1]) # 소비구조증가효과
TCTD = TC1 + TC11
TC111 = np.concatenate((TCTD, TC1, TC11), axis=1)
# TC111

In [ ]:
TI1 = ((RD1@(np.eye(32)-M1))@GDI2).reshape([32,1])  # 소비규모증가효과
TI11 = ((RD1@(np.eye(32)-M1))@GDI3).reshape([32,1]) # 소비구조증가효과
TITD = TI1 + TI11
TI111 = np.concatenate((TITD, TI1, TI11), axis=1)

In [ ]:
TQ = TC1+TI1 # 국내최종수요규모증가효과
TS = TC11+TI11 # 국내최종수요규조변화효과
TQS = TQ+TS
T1 = np.concatenate((TQS, TQ, TS), axis=1)
T1.shape

(32, 3)

In [ ]:
T2 = (RD1@ED2).reshape([32,1]) # 수출규모증가효과
T22 = (RD1@ED3) .reshape([32,1])# 수출구조변화효과
TTE = T2+T22
T2 = np.concatenate((TTE, T2, T22), axis=1)

In [ ]:
T3 = ((RD1@(M1 - M0))@DF0).reshape([32,1]) # 최종재수입대체효과
T3 = T3*-1
T4 = ((RD1@(AM1 - II0)))@C0[0:32,] # 중간재수입대체효과
T4 = T4*-1
T34 = T3 + T4
TI = np.concatenate((T34, T3, T4), axis=1)
TI



array([[ -3016222.634,  -1896596.938,  -1119625.696],
       [    96765.941,    -19072.825,    115838.766],
       [ -3667041.171,  -4007564.783,    340523.612],
       [-12026264.79 ,  -6350206.128,  -5676058.663],
       [ -1544073.497,   -358618.018,  -1185455.48 ],
       [  1448801.495,   -874896.37 ,   2323697.864],
       [ -9629837.821,  -6965931.58 ,  -2663906.241],
       [  -860832.462,   -540370.542,   -320461.919],
       [  2397882.26 ,    514512.52 ,   1883369.74 ],
       [  -107976.446,    -76589.539,    -31386.908],
       [ -9282714.736,   2915429.416, -12198144.152],
       [ -3992999.929,   -559583.302,  -3433416.627],
       [ -1673506.852,   3209846.001,  -4883352.853],
       [ -1278584.563,  -4189708.236,   2911123.673],
       [ -1796537.659,   -841959.102,   -954578.557],
       [   275791.836,  -1057942.403,   1333734.239],
       [ -1219641.517,   -669846.556,   -549794.96 ],
       [  -170149.327,   -107213.979,    -62935.348],
       [  -242540.222,    -4

In [ ]:
T5 = (RD1@(AA-(II1-AM1)))@C0 # 기술변화효과
T5.shape
T5

array([[  2538814.668],
       [  -189244.495],
       [ 15646198.872],
       [  1704385.787],
       [ -2949241.127],
       [  2115141.913],
       [ -1936106.795],
       [   693113.125],
       [ -6404024.808],
       [  5543489.49 ],
       [  2351453.109],
       [ -4309748.321],
       [ -7654992.741],
       [ 11849473.531],
       [   459872.651],
       [  5620627.169],
       [-10101050.153],
       [ -3352771.255],
       [ -1589234.291],
       [ 16668840.826],
       [  1738486.109],
       [  6933957.062],
       [   167737.281],
       [  -260715.09 ],
       [  5204189.052],
       [  3427543.547],
       [ 12004356.422],
       [  1166810.992],
       [   -24775.696],
       [  3178961.314],
       [ -3358323.034],
       [  1521876.13 ]])

In [ ]:
DCT0 = C1-C0  #산출액 실질변동액
DCT1 = TCTD + TITD + TTE +T3 + T4 + T5 # 총변동요인
DCT = DCT0-DCT1

In [ ]:
SUM = np.concatenate((C0, C1, DCT0, DCT1, DCT, T1, TC111, TI111, T2, TI, T5), axis=1)
SUM.shape
SUM1 = np.sum(SUM, axis=0).reshape([1,21])
PGD1015 = (np.concatenate((SUM, SUM1), axis=0))
PGD1015_f = (PGD1015/1000).round(1)

In [ ]:
PGD1015_ff = pd.DataFrame(PGD1015_f,columns = ['10년산출액', '15년산출액', '15-10산출액(A)', '요인별합계(B)', 'B-A', '국내최종수요증감액', '국내최종수요(규모증가)', '국내최종수요(구조변화)',
                                                     '소비', '소비(규모)','소비(구조)','투자', '투자(규모)','투자(구조)', '수출', '수출(규모)','수출(구조)', '수입', '최종재수입','중간재수입', '기술변화'],

                         index=['농림수산품', '광산품', '음식료품', '섬유 및 가죽제품', '목재 및 종이, 인쇄', ' 석탄 및 석유제품', '화학제품', '비금속광물제품', '1차금속제품', '금속가공제품',
                                '컴퓨터, 전자 및 광학기기', '전기장비', '기계 및 장비', '운송장비', '기타제조업제품', '제조임가공 및 산업용 장비 수리', '전력, 가스 및 증기', '수도, 폐기물처리 및 재활용 서비스',
                                '건설', '도소매', '운송서비스', '음석점및숙박', '정보통신 및 방송서비스', '금융및보험서비스', '부동산서비스', '전문, 과학 및 기술서비스',
                                '사업지원서비스', '공공행정, 국방 및 사회보장', '교육서비스', '보건 및 사회복지서비스', '예술, 스포츠 및 여가 관련 서비스', '기타 서비스', '계'])  # pandas로 내보내기

In [ ]:
PGD1015_ff

,10년산출액,15년산출액,15-10산출액(A),요인별합계(B),B-A,국내최종수요증감액,국내최종수요(규모증가),국내최종수요(구조변화),소비,소비(규모),...,투자,투자(규모),투자(구조),수출,수출(규모),수출(구조),수입,최종재수입,중간재수입,기술변화
농림수산품,59105.4,61371.7,2266.3,2266.3,0.0,1709.1,7086.2,-5377.1,3261.4,6564.0,...,-1552.3,522.2,-2074.5,1034.6,2283.0,-1248.3,-3016.2,-1896.6,-1119.6,2538.8
광산품,3805.0,4125.2,320.3,320.3,0.0,57.5,248.2,-190.8,58.4,67.6,...,-0.9,180.6,-181.6,355.3,392.0,-36.7,96.8,-19.1,115.8,-189.2
음식료품,106663.7,122147.2,15483.5,15483.5,0.0,466.3,14229.7,-13763.4,3686.2,13465.7,...,-3219.9,764.0,-3983.9,3038.0,5328.3,-2290.3,-3667.0,-4007.6,340.5,15646.2
섬유 및 가죽제품,72748.2,79329.1,6580.9,6580.9,-0.0,181.2,4240.3,-4059.0,1537.1,3779.8,...,-1355.8,460.5,-1816.3,16721.6,10380.4,6341.2,-12026.3,-6350.2,-5676.1,1704.4
"목재 및 종이, 인쇄",40939.6,42914.0,1974.4,1974.4,-0.0,3797.7,3002.2,795.5,2422.8,2122.3,...,1374.9,880.0,495.0,2670.0,3881.4,-1211.4,-1544.1,-358.6,-1185.5,-2949.2
석탄 및 석유제품,88115.6,110150.0,22034.4,22034.4,0.0,1857.2,5754.6,-3897.4,2667.4,4770.2,...,-810.2,984.4,-1794.6,16613.3,15889.0,724.3,1448.8,-874.9,2323.7,2115.1
화학제품,206672.6,245383.2,38710.6,38710.6,0.0,9380.4,9449.1,-68.7,7873.3,6856.5,...,1507.1,2592.6,-1085.5,40896.1,39657.0,1239.1,-9629.8,-6965.9,-2663.9,-1936.1
비금속광물제품,35071.6,39678.7,4607.1,4607.1,-0.0,1364.7,2488.8,-1124.1,529.2,470.1,...,835.5,2018.7,-1183.2,3410.0,2829.1,580.9,-860.8,-540.4,-320.5,693.1
1차금속제품,115463.4,124914.2,9450.8,9450.8,0.0,-2216.6,3918.8,-6135.4,1358.0,1196.2,...,-3574.6,2722.6,-6297.2,15673.6,24246.1,-8572.6,2397.9,514.5,1883.4,-6404.0
금속가공제품,81461.3,100287.6,18826.2,18826.2,0.0,4145.0,4899.4,-754.5,1997.0,1837.2,...,2147.9,3062.2,-914.3,9245.8,13110.8,-3865.1,-108.0,-76.6,-31.4,5543.5


In [ ]:
PGD1015_ff.to_csv('산출액변동요인.csv', encoding="utf-8-sig")

In [ ]:
Ctbr =  PGD1015[:,5:]
Ctbr.shape

(33, 16)

In [ ]:
sm_gap = PGD1015[:,3].reshape([33,1])  # 요인별 합계
sm_gap.shape

(33, 1)

In [ ]:
sm_gap1 = np.tile(sm_gap, reps=[1,16]) # 요인별 합계를 33x16으로 반복
sm_gap1.shape

(33, 16)

In [ ]:
Ctrb = (Ctbr/sm_gap1)*100  # 각 항목별 기여율
Ctrb_f = Ctrb.round(1)

In [ ]:
Ctrb_ff = pd.DataFrame(Ctrb_f,columns = ['국내최종수요증감액', '국내최종수요(규모증가)', '국내최종수요(구조변화)','소비', '소비(규모)','소비(구조)','투자', '투자(규모)','투자(구조)',
                                         '수출', '수출(규모)','수출(구조)', '수입', '최종재수입','중간재수입', '기술변화'],

                         index=['농림수산품', '광산품', '음식료품', '섬유 및 가죽제품', '목재 및 종이, 인쇄', ' 석탄 및 석유제품', '화학제품', '비금속광물제품', '1차금속제품', '금속가공제품',
                                '컴퓨터, 전자 및 광학기기', '전기장비', '기계 및 장비', '운송장비', '기타제조업제품', '제조임가공 및 산업용 장비 수리', '전력, 가스 및 증기', '수도, 폐기물처리 및 재활용 서비스',
                                '건설', '도소매', '운송서비스', '음석점및숙박', '정보통신 및 방송서비스', '금융및보험서비스', '부동산서비스', '전문, 과학 및 기술서비스',
                                '사업지원서비스', '공공행정, 국방 및 사회보장', '교육서비스', '보건 및 사회복지서비스', '예술, 스포츠 및 여가 관련 서비스', '기타 서비스', '계'])  # pandas로 내보내기

In [ ]:
Ctrb_ff.to_csv('항목별 기여율.csv', encoding="utf-8-sig")

In [ ]:
YR = (((PGD1015[:,1]/PGD1015[:,0])**(1/5)-1)*100).reshape([33,1])
YR

array([[ 0.755],
       [ 1.629],
       [ 2.748],
       [ 1.747],
       [ 0.946],
       [ 4.565],
       [ 3.493],
       [ 2.499],
       [ 1.586],
       [ 4.246],
       [ 7.75 ],
       [ 1.629],
       [ 1.346],
       [ 3.545],
       [ 2.735],
       [ 5.84 ],
       [-1.144],
       [-1.31 ],
       [ 0.541],
       [ 5.803],
       [ 1.792],
       [ 4.201],
       [ 4.326],
       [ 3.781],
       [ 3.701],
       [ 4.839],
       [ 6.369],
       [ 4.203],
       [ 1.729],
       [ 5.498],
       [ 0.045],
       [ 3.825],
       [ 3.421]])

In [ ]:
Ctrb_amt = (Ctrb*YR/100)
Ctrb_amt_f = Ctrb_amt.round(3)
Ctrb_amt_f

array([[ 0.57 ,  2.362, -1.792,  1.087,  2.188, -1.101, -0.517,  0.174,
        -0.691,  0.345,  0.761, -0.416, -1.005, -0.632, -0.373,  0.846],
       [ 0.292,  1.263, -0.971,  0.297,  0.344, -0.047, -0.005,  0.919,
        -0.924,  1.808,  1.994, -0.187,  0.492, -0.097,  0.589, -0.963],
       [ 0.083,  2.525, -2.443,  0.654,  2.39 , -1.736, -0.571,  0.136,
        -0.707,  0.539,  0.946, -0.406, -0.651, -0.711,  0.06 ,  2.777],
       [ 0.048,  1.126, -1.078,  0.408,  1.003, -0.595, -0.36 ,  0.122,
        -0.482,  4.439,  2.756,  1.683, -3.193, -1.686, -1.507,  0.452],
       [ 1.821,  1.439,  0.381,  1.161,  1.017,  0.144,  0.659,  0.422,
         0.237,  1.28 ,  1.861, -0.581, -0.74 , -0.172, -0.568, -1.414],
       [ 0.385,  1.192, -0.807,  0.553,  0.988, -0.436, -0.168,  0.204,
        -0.372,  3.442,  3.292,  0.15 ,  0.3  , -0.181,  0.481,  0.438],
       [ 0.847,  0.853, -0.006,  0.711,  0.619,  0.092,  0.136,  0.234,
        -0.098,  3.691,  3.579,  0.112, -0.869, -0.629, -0

In [ ]:
Ctrb_amt_ff = pd.DataFrame(Ctrb_amt_f,columns = ['국내최종수요증감액', '국내최종수요(규모증가)', '국내최종수요(구조변화)','소비', '소비(규모)','소비(구조)','투자', '투자(규모)','투자(구조)',
                                         '수출', '수출(규모)','수출(구조)', '수입', '최종재수입','중간재수입', '기술변화'],

                         index=['농림수산품', '광산품', '음식료품', '섬유 및 가죽제품', '목재 및 종이, 인쇄', ' 석탄 및 석유제품', '화학제품', '비금속광물제품', '1차금속제품', '금속가공제품',
                                '컴퓨터, 전자 및 광학기기', '전기장비', '기계 및 장비', '운송장비', '기타제조업제품', '제조임가공 및 산업용 장비 수리', '전력, 가스 및 증기', '수도, 폐기물처리 및 재활용 서비스',
                                '건설', '도소매', '운송서비스', '음석점및숙박', '정보통신 및 방송서비스', '금융및보험서비스', '부동산서비스', '전문, 과학 및 기술서비스',
                                '사업지원서비스', '공공행정, 국방 및 사회보장', '교육서비스', '보건 및 사회복지서비스', '예술, 스포츠 및 여가 관련 서비스', '기타 서비스', '계'])  # pandas로 내보내기

In [ ]:
Ctrb_amt_ff.to_csv('항목별 기여도.csv', encoding="utf-8-sig")